In [0]:
import json
import random
import datetime
import os
import time
import uuid

class GeradorDadosFakeTelecom:
    planos = ["Pré-pago", "Controle", "Pós-pago"]
    problemas = ["Queda de sinal", "Cobrança incorreta", "Suporte técnico", "Portabilidade", "Configuração de internet"]
    canais = ["Telefone", "Chat", "Loja física", "E-mail"]
    cidades = ["São Paulo", "Rio de Janeiro", "Belo Horizonte", "Curitiba", "Porto Alegre", "Salvador"]

    def __init__(self, base_path, num_clientes=200, num_assinaturas=300, num_cdrs_iniciais=2000):
        self.base_path = base_path
        self.customers_dir = os.path.join(base_path, "clientes")
        self.subs_dir = os.path.join(base_path, "assinaturas")
        self.cdrs_dir = os.path.join(base_path, "cdrs")
        self.tickets_dir = os.path.join(base_path, "tickets")
        self.checkpoint_path = os.path.join(base_path, "checkpoint.json")

        self.num_clientes = num_clientes
        self.num_assinaturas = num_assinaturas
        self.num_cdrs_iniciais = num_cdrs_iniciais

        self.clientes = []
        self.assinaturas = []
        self.ultimo_cdr_id = 0
        self.ultimo_ticket_id = 0

        self.carregar_checkpoint()

    def carregar_checkpoint(self):
        if os.path.exists(self.checkpoint_path):
            with open(self.checkpoint_path, 'r', encoding='utf-8') as f:
                checkpoint = json.load(f)
                self.clientes = checkpoint["clientes"]
                self.assinaturas = checkpoint["assinaturas"]
                self.ultimo_cdr_id = checkpoint["ultimo_cdr_id"]
                self.ultimo_ticket_id = checkpoint["ultimo_ticket_id"]

    def salvar_checkpoint(self):
        checkpoint = {
            "clientes": self.clientes,
            "assinaturas": self.assinaturas,
            "ultimo_cdr_id": self.ultimo_cdr_id,
            "ultimo_ticket_id": self.ultimo_ticket_id
        }
        with open(self.checkpoint_path, 'w', encoding='utf-8') as f:
            json.dump(checkpoint, f)

    def gerar_clientes(self):
        clientes = []
        for i in range(1, self.num_clientes + 1):
            cliente = {
                "id_cliente": str(uuid.uuid4()),
                "nome": f"Cliente_{i}",
                "cidade": random.choice(self.cidades),
                "telefone": f"+55{random.randint(1000000000, 9999999999)}",
                "email": f"cliente{i}@example.com"
            }
            clientes.append(cliente)
        return clientes

    def gerar_assinaturas(self):
        subs = []
        for i in range(1, self.num_assinaturas + 1):
            sub = {
                "id_assinatura": str(uuid.uuid4()),
                "id_cliente": random.choice(self.clientes)["id_cliente"],
                "plano": random.choice(self.planos),
                "mensalidade": round(random.choice([29.9, 49.9, 79.9, 99.9, 149.9]), 2),
                "status": random.choice(["Ativa", "Suspensa", "Cancelada"])
            }
            subs.append(sub)
        return subs

    def gerar_cdrs(self, inicio, fim, prox_id, num_cdrs):
        cdrs = []
        tempo_total = (fim - inicio).total_seconds()
        for _ in range(num_cdrs):
            offset = random.randrange(int(tempo_total)) if tempo_total > 0 else 0
            momento = inicio + datetime.timedelta(seconds=offset)
            tipo = random.choice(["voz", "sms", "dados"])
            duracao = random.randint(1, 3600) if tipo == "voz" else 0
            data_mb = round(random.uniform(0.1, 500.0), 2) if tipo == "dados" else 0
            custo = round(random.uniform(0.0, 5.0), 2)

            cdr = {
                "id_cdr": prox_id,
                "id_assinatura": random.choice(self.assinaturas)["id_assinatura"],
                "timestamp": momento.strftime("%Y-%m-%d %H:%M:%S"),
                "tipo": tipo,
                "duracao_segundos": duracao,
                "dados_mb": data_mb,
                "custo_brl": custo
            }
            cdrs.append(cdr)
            prox_id += 1
        return cdrs, prox_id - 1

    def gerar_tickets(self, num_tickets=10):
        tickets = []
        for _ in range(num_tickets):
            self.ultimo_ticket_id += 1
            ticket = {
                "id_ticket": self.ultimo_ticket_id,
                "id_cliente": random.choice(self.clientes)["id_cliente"],
                "aberto_em": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "problema": random.choice(self.problemas),
                "canal": random.choice(self.canais),
                "status": random.choice(["Aberto", "Resolvido", "Fechado"])
            }
            tickets.append(ticket)
        return tickets

    def carga_inicial(self):
        os.makedirs(self.customers_dir, exist_ok=True)
        os.makedirs(self.subs_dir, exist_ok=True)
        os.makedirs(self.cdrs_dir, exist_ok=True)
        os.makedirs(self.tickets_dir, exist_ok=True)

        self.clientes = self.gerar_clientes()
        self.assinaturas = self.gerar_assinaturas()

        fim = datetime.datetime.now() - datetime.timedelta(days=1)
        inicio = fim - datetime.timedelta(days=30)
        cdrs, self.ultimo_cdr_id = self.gerar_cdrs(inicio, fim, 1, self.num_cdrs_iniciais)

        with open(os.path.join(self.customers_dir, "clientes_inicial.json"), 'w', encoding='utf-8') as f_c:
            for c in self.clientes:
                f_c.write(json.dumps(c) + "\n")

        with open(os.path.join(self.subs_dir, "assinaturas_inicial.json"), 'w', encoding='utf-8') as f_s:
            for s in self.assinaturas:
                f_s.write(json.dumps(s) + "\n")

        with open(os.path.join(self.cdrs_dir, "cdrs_inicial.json"), 'w', encoding='utf-8') as f_cdr:
            for cdr in cdrs:
                f_cdr.write(json.dumps(cdr) + "\n")

        tickets = self.gerar_tickets(20)
        with open(os.path.join(self.tickets_dir, "tickets_inicial.json"), 'w', encoding='utf-8') as f_t:
            for t in tickets:
                f_t.write(json.dumps(t) + "\n")

    def gerar_incremental(self, novos_cdrs=50, novos_tickets=5):
        agora = datetime.datetime.now()
        uma_hora_atras = agora - datetime.timedelta(hours=1)

        novos_cdrs_lista, novo_ultimo_id = self.gerar_cdrs(uma_hora_atras, agora, self.ultimo_cdr_id + 1, novos_cdrs)
        self.ultimo_cdr_id = novo_ultimo_id

        novos_tickets_lista = self.gerar_tickets(novos_tickets)

        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

        with open(os.path.join(self.cdrs_dir, f"cdrs_incremental_{timestamp}.json"), 'w', encoding='utf-8') as f_cdr:
            for c in novos_cdrs_lista:
                f_cdr.write(json.dumps(c) + "\n")

        with open(os.path.join(self.tickets_dir, f"tickets_incremental_{timestamp}.json"), 'w', encoding='utf-8') as f_t:
            for t in novos_tickets_lista:
                f_t.write(json.dumps(t) + "\n")

    def verificar_e_gerar_dados(self):
        if not os.path.exists(self.cdrs_dir) or not os.listdir(self.cdrs_dir):
            print("Diretório vazio ou inexistente. Gerando carga inicial...")
            self.carga_inicial()
        else:
            print("Dados existentes. Gerando carga incremental...")
            self.gerar_incremental()
        self.salvar_checkpoint()

    def stream(self, num_lotes, intervalo=15):
        for lote in range(num_lotes):
            print(f"Gerando lote {lote+1}/{num_lotes}...")
            self.gerar_incremental()
            self.salvar_checkpoint()
            time.sleep(intervalo)


# Exemplo de uso
if __name__ == "__main__":
    gerador = GeradorDadosFakeTelecom(base_path="telecom_data", num_clientes=100, num_assinaturas=150, num_cdrs_iniciais=1000)
    gerador.verificar_e_gerar_dados()
    gerador.stream(num_lotes=3, intervalo=5)
